In [ ]:
import cv2
import numpy as np

# Open a video file
video_path = "D:\MAPUA\Thesis\Thoracic Rotation in Quadruped - Ask Doctor Jo - Trim.mp4"  # Replace with the path to your video file
cap = cv2.VideoCapture(video_path)

# Initialize the Farneback optical flow parameters
farneback_params = {
    'pyr_scale': 0.5,
    'levels': 5,
    'winsize': 11,
    'iterations': 5,
    'poly_n': 5,
    'poly_sigma': 1.1,
    'flags': 0
}

# Read the first frame
ret, prev_frame = cap.read()
prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

while cap.isOpened():
    # Read the current frame
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the current frame to grayscale
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculate optical flow using Farneback method
    flow = cv2.calcOpticalFlowFarneback(prev_frame_gray, frame_gray, None, **farneback_params)

    # Display the optical flow
    h, w = flow.shape[:2]
    y, x = np.mgrid[0:h, 0:w]
    indices = np.column_stack((x.flatten(), y.flatten()))
    displacements = flow.reshape(-1, 2)

    for (dx, dy), (x, y) in zip(displacements, indices):
        cv2.circle(frame, (x, y), 1, (0, 255, 0), -1)
        cv2.line(frame, (x, y), (int(x + dx), int(y + dy)), (0, 255, 0), 1)

    # Display the result
    cv2.imshow('Optical Flow', frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

    # Set the current frame as the previous frame for the next iteration
    prev_frame_gray = frame_gray

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()
